In [1]:
from collections import defaultdict

import numpy as np
import pandas as pd

In [2]:
#get answers from csv
a_dict = dict()
answers = pd.read_csv('pretrainingkey.csv')
questions = answers.loc[:,"Example Scenario in Survey"]
correct = answers.loc[:,"Correct Answer"]
for i in range(0, len(questions)):
    cq = str(questions[i]).strip()
    ca = str(correct[i]).strip()
    a_dict[cq] = str(ca)


In [11]:
#dataframe raw responses
df = pd.read_csv('pretraining.csv')
#optional questions should not be collected
ignore = {"Q161", "Q227", "Q202"}
timings = set()
pretraining = set()
neededTimings = set()
dateDict = dict()
for j in range(1, len(df.iloc[:, 0])):
        participantId = df.iat[j, df.columns.get_loc("Q162")]
        date = df.iat[j, df.columns.get_loc("StartDate")]
        if "08" in date:
            dateDict[participantId] = 1
        else:
            dateDict[participantId] = 2
#drop QID row
df = df.drop(labels=1, axis=0)
#descriptions for each question
descriptionsDict = dict()
start = df.columns.get_loc("Q1-1-2")
end = df.columns.get_loc("Q298")
for i in range(start, end + 1, 1):
    key = df.columns[i]
    if "Click" in key or "Submit" in key:
        if "Submit" in key:
            neededTimings.add(key)
        timings.add(key)
        continue
    if key.startswith("Q1-") or key.startswith("Q2-"):
        pretraining.add(key)
    if df.iat[0,i].startswith("Correct") or df.iat[0,i].startswith("Incorrect") or "Thank you" in df.iat[0, i]:
        continue
    if key not in ignore:
        descriptionsDict[key] = df.iat[0, i]
print(descriptionsDict)

{"ImportId":"startDate","timeZone":"America/Denver"}
2022-08-25 12:58:26
2022-08-25 12:58:26
2022-08-25 12:57:30
2022-08-25 12:59:30
2022-08-25 12:58:08
2022-08-25 12:57:46
2022-08-25 12:58:17
2022-08-25 12:57:46
2022-08-25 12:59:23
2022-08-25 12:59:09
2022-08-25 13:15:38
2022-08-25 13:29:59
2022-08-27 12:07:54
2022-08-27 12:06:53
2022-08-27 12:08:17
2022-08-27 12:07:19
2022-08-27 12:08:20
2022-08-27 12:05:46
2022-08-27 12:06:10
2022-08-27 12:07:05
2022-08-27 12:17:25
2022-08-27 12:18:37
2022-08-27 12:24:55
2022-08-27 12:12:40
2022-08-27 16:21:02
2022-08-27 16:17:42
2022-08-27 16:18:23
2022-08-27 16:17:54
2022-08-27 16:28:03
2022-08-27 16:26:24
2022-08-27 16:18:18
2022-08-27 16:38:46
2022-08-27 16:40:18
{'Q1-1-2': 'Which of the following instances of webview usage should be reported as data sharing? Please select all that apply.', 'Q1-2-2': 'Which of the following instances of webview usage should be disclosed as  data collection? Please select all that apply.', 'Q1-5-2': 'Which of the

In [4]:
#categorize questions by case
categoryDict = dict()
groupDict = dict()
sectionDict = dict()
for question in descriptionsDict:
    if question.startswith("Q1-") or question.startswith("Q2-"):
        sectionDict[question] = "Pretraining"
    else:
        sectionDict[question] = "Main Survey"
    if sectionDict[question] != "Pretraining":
        if "Our app accesses GPS location and sends it to our server." in descriptionsDict[question]:
             categoryDict[question] = "collect basic"
             groupDict[question] = "Collect Basic"
        elif "Our app accesses GPS location and sends it to a third party server." in descriptionsDict[question]:
            categoryDict[question] = "share basic"
            groupDict[question] = "Share Basic"
        elif "users to navigate the open web via a webview. When the user opens google.com, it accesses the user's GPS location and sends the GPS location to Google's server" in descriptionsDict[question] or "Our app accesses and processes the user's GPS location directly on the user’s device. The data never leaves the device" in descriptionsDict[question] or "Our app encrypts the user’s message before it is sent to another user, where it is decrypted" in descriptionsDict[question]:
            categoryDict[question] = "collect exempt"
            if "users to navigate the open web via a webview. When the user opens google.com, it accesses the user's GPS location and sends the GPS location to Google's server" in descriptionsDict[question]:
                groupDict[question] = "Webviews"
            elif "Our app accesses and processes the user's GPS location directly on the user’s device. The data never leaves the device" in descriptionsDict[question]:
                groupDict[question] = "On-device processing and transfers"
            elif "Our app encrypts the user’s message before it is sent to another user, where it is decrypted" in descriptionsDict[question]:
                groupDict[question] = "End to End Encryption"
        elif "Our app allows users to navigate the open web via a webview. When the user opens google.com, it accesses the user's GPS location and sends the GPS location to a third party server" in descriptionsDict[question] or "Our app accesses GPS location and sends it off the device to Google Maps to get real-time traffic updates" in descriptionsDict[question] or "Our app accesses GPS location and stores it on Google's server using Google Cloud Storage" in descriptionsDict[question] or "Our app accesses the user’s GPS location and sends it to government agencies in response to a legal request" in descriptionsDict[question] or "Our app's user explicitly initiates an action that sends their full name to a third party server" in descriptionsDict[question] or "Our app sends the user's full name to a third party server after obtaining the user's consent to it" in descriptionsDict[question] or "Our app sends the user's medical records that have been fully anonymized to a third party server." in descriptionsDict[question]:
            categoryDict[question] = "share exempt"
            if "Our app allows users to navigate the open web via a webview. When the user opens google.com, it accesses the user's GPS location and sends the GPS location to a third party server" in descriptionsDict[question]:
                groupDict[question] = "Webviews"
            elif "Our app accesses GPS location and sends it off the device to Google Maps to get real-time traffic updates" in descriptionsDict[question] or "Our app accesses GPS location and stores it on Google's server using Google Cloud Storage" in descriptionsDict[question]:
                groupDict[question] = "Service Providers"
            elif "Our app accesses the user’s GPS location and sends it to government agencies in response to a legal request" in descriptionsDict[question]:
                groupDict[question] = "Legal Request"
            elif "Our app's user explicitly initiates an action that sends their full name to a third party server" in descriptionsDict[question] or "Our app sends the user's full name to a third party server after obtaining the user's consent to it" in descriptionsDict[question]:
                groupDict[question] = "User Consent"
            elif "Our app sends the user's medical records that have been fully anonymized to a third party server." in descriptionsDict[question]:
                groupDict[question] = "Anonymization"
        elif "Our app accesses GPS location and sends it to our server via libraries and/or SDKs included in our app" in descriptionsDict[question] or "website then accesses the user's GPS location through the webview, and sends it to our server" in descriptionsDict[question] or "Our app sends the user's GPS location to our server to search for nearby places but does not store the GPS location after serving the request" in descriptionsDict[question] or "Our app requests for the user's full name and only sends the pseudonymized name to our server" in descriptionsDict[question] or "Our app encrypts the user’s bank account information and sends it to our server" in descriptionsDict[question] or "Our app asks for a user's birthday for marketing communication, but that info is not required" in descriptionsDict[question]:
            categoryDict[question] = "collect special"
            if "Our app accesses GPS location and sends it to our server via libraries and/or SDKs included in our app" in descriptionsDict[question]:
                groupDict[question] = "Libraries/SDKs"
            elif "website then accesses the user's GPS location through the webview, and sends it to our server" in descriptionsDict[question]:
                groupDict[question] = "Webviews"
            elif "Our app sends the user's GPS location to our server to search for nearby places but does not store the GPS location after serving the request" in descriptionsDict[question]:
                groupDict[question] = "Ephemeral Processing"
            elif "Our app requests for the user's full name and only sends the pseudonymized name to our server" in descriptionsDict[question]:
                groupDict[question] = "Pseudynoymization"
            elif "Our app encrypts the user’s bank account information and sends it to our server" in descriptionsDict[question]:
                groupDict[question] = "Encryption"
            elif "Our app asks for a user's birthday for marketing communication, but that info is not required" in descriptionsDict[question]:
                groupDict[question] = "Optional Data Collection"
        else:
            categoryDict[question] = "share special"
            if "We send the user's GPS location stored on our server to a third party server" in descriptionsDict[question]:
                groupDict[question] = "Off-device Server Transfers"
            elif "sends it to another app directly on the device" in descriptionsDict[question]:
                groupDict[question] = "On-device processing and transfers"
            elif "Our app accesses GPS location and sends it directly to a third party server via libraries and/or SDKs included in our app" in descriptionsDict[question]:
                groupDict[question] = "Libraries/SDKs"
            elif "website accesses the GPS location through the webview, and sends it to a third party server" in descriptionsDict[question]:
                groupDict[question] = "Webviews"
            else:
                print(descriptionsDict[question])

    else:
        if "sharing" in descriptionsDict[question]:
                categoryDict[question] = "share pretraining"
        elif "collection" in descriptionsDict[question]:
             categoryDict[question] = "collect pretraining"
        if question.startswith("Q1"):
            categoryDict[question] = str(categoryDict[question] + " 1")
        if question.startswith("Q2"):
            categoryDict[question] = str(categoryDict[question] + " 2")
        if "webview" in descriptionsDict[question]:
            groupDict[question] = "Webviews"
        elif "service provider" in descriptionsDict[question] or "navigation API" in descriptionsDict[question]:
            groupDict[question] = "Service Providers"
        elif "optional data handling" in descriptionsDict[question] or "optionally provide" in descriptionsDict[question]:
            groupDict[question] = "Optional Data Collection"
        elif "anonymized" in descriptionsDict[question]:
            groupDict[question] = "Anonymization"
        elif "user action" in descriptionsDict[question] or "weather API" in descriptionsDict[question]:
            groupDict[question] = "User Consent"
        elif "following instances of data usage" in descriptionsDict[question] or "analytics reasons" in descriptionsDict[question]:
            groupDict[question] = "Legal Request"
        else:
            print(descriptionsDict[question])
print(categoryDict)


This is an attention check. Please go ahead and select the option that begins with "N".
{'Q1-1-2': 'share pretraining 1', 'Q1-2-2': 'collect pretraining 1', 'Q1-5-2': 'share pretraining 1', 'Q1-6-2': 'collect pretraining 1', 'Q1-7-2': 'share pretraining 1', 'Q1-8-2': 'share pretraining 1', 'Q1-9-2': 'share pretraining 1', 'Q2-1-1': 'share pretraining 2', 'Q2-2-1': 'collect pretraining 2', 'Q2-5-1': 'share pretraining 2', 'Q2-6-1': 'collect pretraining 2', 'Q2-7-1': 'share pretraining 2', 'Q2-8-1': 'share pretraining 2', 'Q2-9-1': 'share pretraining 2', 'Q203': 'share special', 'Q205': 'share basic', 'Q278': 'collect basic', 'Q279': 'share special', 'Q280': 'share special', 'Q281': 'collect special', 'Q282': 'share special', 'Q283': 'collect exempt', 'Q285': 'collect special', 'Q286': 'collect exempt', 'Q287': 'share special', 'Q288': 'share exempt', 'Q289': 'share exempt', 'Q290': 'share exempt', 'Q291': 'share exempt', 'Q292': 'collect exempt', 'Q293': 'collect special', 'Q294': 'coll

In [5]:
#get correct answers for each question (based on ID)
correctAnswerDict = dict()
#question names
nameDict = dict()
for question in descriptionsDict:
    search_key = descriptionsDict[question]
    if "Data usage scenario:" in search_key:
        search_key = search_key.replace("Data usage scenario:", "")
    if "Is" in search_key:
        end = search_key.index("Is")
        search_key = search_key[1:end-1].strip()
    else:
        end = len(search_key)
        search_key = search_key[0:end].strip()
    if search_key[-1] == ".":
        search_key = search_key[:-1]
    if search_key in a_dict:
        print(search_key, question)
        correctAnswerDict[question] = a_dict[search_key]
        nameDict[question] = search_key.replace("\n", "")
    else:
        for key in a_dict:
            if search_key in key:
                    correctAnswerDict[question] = a_dict[key]
                    break
        nameDict[question] = key.replace("\n", "")
    if "Please" in nameDict[question]:
        nameDict[question] = nameDict[question].replace("Please" , " Please")
print(correctAnswerDict["Q2-5-1"])

An app opens a webview that allows users to freely search google.com and google.com sends user data to a third party server. Does it need to be disclosed as data sharing? Q2-1-1
An app opens a webview that allows users to freely search bing.com which sends user data to Bing's server. Does it need to be disclosed as data collection? Q2-2-1
A GPS app sends user location data off-device to a navigation API, which helps determine the best route as per the app's given instructions. Does it need to be disclosed as data sharing? Q2-5-1
An app asks the user to optionally provide their address for mailing communications and sends this data to a third party server. Does it need to be disclosed as data collection? Q2-6-1
An app sends partially anonymized information regarding the user's political orientation to a third party server. This data is able to be re-associated with the user if additional information is provided. Does it need to be disclosed as data sharing? Q2-7-1
An app sends the user'

In [6]:
#potential answers, for indexing
collection = ['Collected', 'Shared', 'Both collected and shared', 'Neither collected nor shared']

attentionChecks = {"Q203"}
answerDict = defaultdict(str)
indexGivenDict = defaultdict(str)
indexExpectedDict = defaultdict(str)
rowIds = []
start = df.columns.get_loc("Q1-1-2")
end = df.columns.get_loc("Q298")
question1Count = dict()
pretrainingCount = dict()
remove = set()
count = 0
start = df.columns.get_loc("Q1-1-2")
end = df.columns.get_loc("Q298")
timingsDict = dict()
for i in range(start, end+1, 1):
    for j in range(1, len(df.iloc[:, 0])):
        participantId = df.iat[j, df.columns.get_loc("Q162")]
        questionId = df.columns[i]
        timingsDict[(participantId, questionId)] = "nan"
        if questionId in ignore or questionId in timings:
            continue
        if df.iat[0,i].startswith("Correct") or df.iat[0,i].startswith("Incorrect") or "Thank you" in df.iat[0, i]:
            continue
        rowId = (participantId, questionId)
        if questionId not in attentionChecks:
            rowIds.append(rowId)
        answer = str(df.iat[j, i])
        if answer == "nan":
            answerDict[rowId] = ""
            continue
        answer = str(answer).strip()
        if questionId not in pretraining:
            for word in collection:
                    if answer[0:6] in word:
                        answer = word
        answerDict[rowId] = str(answer)
        #remove questions that don't pass the attentionChecks
        if questionId in attentionChecks:
            if answer != correctAnswerDict[questionId]:
                remove.add(participantId)
                print(participantId)
                print(answer)
                print(questionId)
        if questionId not in pretraining:
            question1Count[participantId] = question1Count.get(participantId, 0) + 1
            index_given = collection.index(answer)
            index_expected = collection.index(correctAnswerDict[questionId])
        else:
            pretrainingCount[participantId] = pretrainingCount.get(participantId, 0) + 1
            index_given = -1
            index_expected = -1

        #look at difference between expected index and given index
        indexGivenDict[rowId] = index_given
        indexExpectedDict[rowId] = index_expected
#make sure user answered required amount of questions
for participant in question1Count:
    if question1Count[participant] < 11:
        print("incomplete question type 1:", participant)
        print(question1Count[participant])
        remove.add(participant)

In [7]:
#make sure user answered all eligibility questions
conditionDict = dict()
for j in range(1, len(df.iloc[:, 0])):
    participantId = df.iat[j, df.columns.get_loc("Q162")]
    ageQuestion = df.columns.get_loc("Q4")
    expQuestion = df.columns.get_loc("Q102")
    understanding = df.columns.get_loc("Q116")
    timingsDict[(participantId, ageQuestion)] = timingsDict[(participantId, expQuestion)] = timingsDict[(participantId, understanding)] = df.iat[j, df.columns.get_loc("Q1165_Page Submit")]
    if str(df.iat[j, ageQuestion]) != "Yes" or str(df.iat[j, expQuestion]) != "Yes" or str(df.iat[j, understanding]) != "Yes":
        print(str(df.iat[j, ageQuestion]))
        print(str(df.iat[j, expQuestion]))
        remove.add(participantId)
        print(participantId)
    if str(df.iat[j, df.columns.get_loc("Q1141")]) != "Yes":
        conditionDict[participantId] = 1
    else:
        conditionDict[participantId] = 0


Yes
No
5ec0ba5fe70ab737cc37559c


In [8]:
for j in range(1, len(df.iloc[:, 0])):
    participantId = df.iat[j, df.columns.get_loc("Q162")]
    question1 = {"Q1-1-2","Q1-1-3","Q1-1-4"}
    for question in question1:
        timingsDict[(participantId, question)] = df.iat[j, df.columns.get_loc("Q1166_Page Submit")]
    question2 = {"Q1-2-2", 	"Q1-2-3","Q1-2-4"}
    for question in question2:
        timingsDict[(participantId, question)] = df.iat[j, df.columns.get_loc("Q1167_Page Submit")]
    question3 = {"Q1-5-2",	"Q1-5-3",	"Q1-5-4"}
    for question in question3:
        timingsDict[(participantId, question)] = df.iat[j, df.columns.get_loc("Q1168_Page Submit")]
    question4 = {"Q1-6-2",	"Q1-6-3",	"Q1-6-4"}
    for question in question4:
        timingsDict[(participantId, question)] = df.iat[j, df.columns.get_loc("Q1169_Page Submit")]
    question5 = {"Q1-7-2",	"Q1-7-3",	"Q1-7-4"}
    for question in question5:
        timingsDict[(participantId, question)] = df.iat[j, df.columns.get_loc("Q1170_Page Submit")]
    question6 = {"Q1-8-2",	"Q1-8-3",	"Q1-8-4"}
    for question in question6:
        timingsDict[(participantId, question)] = df.iat[j, df.columns.get_loc("Q1171_Page Submit")]
    question7 = {"Q1-9-2",	"Q1-9-3",	"Q1-9-4"}
    for question in question7:
        timingsDict[(participantId, question)] = df.iat[j, df.columns.get_loc("Q1172_Page Submit")]
    question8 = {"Q2-1-1",	"Q2-1-2",	"Q2-1-3"}
    for question in question8:
        timingsDict[(participantId, question)] = df.iat[j, df.columns.get_loc("Q1173_Page Submit")]
    question10 = {"Q2-2-1",	"Q2-2-2",	"Q2-2-3"}
    for question in question10:
        timingsDict[(participantId, question)] = df.iat[j, df.columns.get_loc("Q1174_Page Submit")]
    question11 = {"Q2-5-1",	"Q2-5-2",	"Q2-5-3"}
    for question in question11:
        timingsDict[(participantId, question)] = df.iat[j, df.columns.get_loc("Q1175_Page Submit")]
    question12 = {"Q2-6-1",	"Q2-6-2",	"Q2-6-3"}
    for question in question12:
        timingsDict[(participantId, question)] = df.iat[j, df.columns.get_loc("Q1176_Page Submit")]
    question13 = {"Q2-7-1",	"Q2-7-2",	"Q2-7-3"}
    for question in question13:
        timingsDict[(participantId, question)] = df.iat[j, df.columns.get_loc("Q1177_Page Submit")]
    question14 = {"Q2-8-1",	"Q2-8-2",	"Q2-8-3"}
    for question in question14:
        timingsDict[(participantId, question)] = df.iat[j, df.columns.get_loc("Q1178_Page Submit")]
    question15 = {"Q2-9-1",	"Q2-9-2",	"Q2-9-3"}
    for question in question15:
        timingsDict[(participantId, question)] = df.iat[j, df.columns.get_loc("Q1179_Page Submit")]

In [9]:
#generating dataframe columns
results = pd.DataFrame()
participantIds = [p for p, q in rowIds]
questionIds = [q for p, q in rowIds]
descriptions = [descriptionsDict[q] for q in questionIds]
categories = [categoryDict[q] for q in questionIds]
groups = [groupDict[q] for q in questionIds]
answers = [answerDict[(p, q)] for p, q in zip(participantIds, questionIds)]
correctAnswers = [correctAnswerDict[q] for q in questionIds]
givenIndices = [indexGivenDict[(p, q)] for p, q in zip(participantIds, questionIds)]
expectedIndices = [indexExpectedDict[(p, q)] for p, q in zip(participantIds, questionIds)]
names = [nameDict[q] for q in questionIds]
dates = [dateDict[p] for p in participantIds]
conditions = [conditionDict[p] for p in participantIds]
timings = [timingsDict[(p, q)] for p, q in rowIds]
results['Question ID'] = questionIds
results['Participant ID'] = participantIds
results['Description'] = descriptions
results['Answer'] = answers
results['Correct Answer'] = correctAnswers
results['Prompt Condition'] = conditions
results["Question Category"] = categories
results["Concept Group"] = groups
results['Question Name'] = names
results['Prompt Condition'] = conditions
results['Answer Index'] = givenIndices
results['Correct Answer Index'] = expectedIndices
results["Timings"] = timings
results["Date Category"] = dates

# filter out rows with empty answers (i.e., not selected in the study)
#remove rows that don't pass the attentionChecks
#results['Correctness'] = (results['Answer'] == results['Correct Answer']).apply(lambda v: 1 if v else 0)
correctness = []
for i in range(len(questionIds)):
    question = questionIds[i]
    if question.startswith("Q2-") and answers[i] == "":
        correctness.append(-1)
    elif answers[i] == correctAnswers[i]:
        correctness.append(1)
    else:
        correctness.append(0)
results["Correctness"] = correctness
# filter out rows with empty answers (i.e., not selected in the study)
for i in range(len(results)):
    if answers[i] == "" and correctness[i] != -1:
        results = results.drop(i)
results = results[~results["Participant ID"].isin(remove)]


In [10]:
results.to_csv("pretrainingOutput.csv")